In [1]:
from os import listdir
import os, random, copy
from PIL import Image
import numpy as np
from collections import defaultdict

In [2]:
def load_data(data_dir="./aligned/"):
	""" Load all PNG images stored in your data directory into a list of NumPy
	arrays.

	Args:
		data_dir: The relative directory path to the CK+ image directory.
	Returns:
		images: A dictionary with keys as emotions and a list containing images associated with each key.
		cnt: A dictionary that stores the # of images in each emotion
	"""
	images = defaultdict(list)

	# Get the list of emotional directory:
	for e in listdir(data_dir):
		# excluding any non-directory files
		if not os.path.isdir(os.path.join(data_dir, e)):
			continue
		# Get the list of image file names
		all_files = listdir(os.path.join(data_dir, e))

		for file in all_files:
			# Load only image files as PIL images and convert to NumPy arrays
			if '.png' in file:
				img = Image.open(os.path.join(data_dir, e, file))
				images[e].append(np.array(img))

	print("Emotions: {} \n".format(list(images.keys())))

	cnt = defaultdict(int)
	for e in images.keys():
		print("{}: {} # of images".format(e, len(images[e])))
		cnt[e] = len(images[e])
	return images, cnt

def balanced_sampler(dataset, cnt, emotions):
	# this ensures everyone has the same balanced subset for model training, don't change this seed value
	random.seed(20)
	print("\nBalanced Set:")
	min_cnt = min([cnt[e] for e in emotions])
	balanced_subset = defaultdict(list)
	for e in emotions:
		balanced_subset[e] = copy.deepcopy(dataset[e])
		random.shuffle(balanced_subset[e])
		balanced_subset[e] = balanced_subset[e][:min_cnt]
		print('{}: {} # of images'.format(e, len(balanced_subset[e])))
	return balanced_subset

def display_face(img):
	""" Display the input image and optionally save as a PNG.

	Args:
		img: The NumPy array or image to display

	Returns: None
	"""
	# Convert img to PIL Image object (if it's an ndarray)
	if type(img) == np.ndarray:
		print("Converting from array to PIL Image")
		img = Image.fromarray(img)

	# Display the image
	img.show()

In [3]:
# example on how to use it
if __name__ == '__main__':
	# The relative path to your image directory
	data_dir = "./PA1/aligned/"
	dataset, cnt = load_data(data_dir)
	# test with happiness and anger
	images = balanced_sampler(dataset, cnt, emotions=['fear','happiness'])
	display_index = 0
	display_face(images['fear'][display_index])

Emotions: ['fear', 'surprise', 'sadness', 'happiness', 'anger', 'disgust'] 

fear: 25 # of images
surprise: 83 # of images
sadness: 28 # of images
happiness: 69 # of images
anger: 45 # of images
disgust: 59 # of images

Balanced Set:
fear: 25 # of images
happiness: 25 # of images
Converting from array to PIL Image


In [5]:
#get image categories
images.keys()

dict_keys(['fear', 'happiness'])

In [6]:
#map categories to numeric categories
d = dict([(y,x) for x,y in enumerate(sorted(set(images.keys())))])

In [7]:
#get X feature array and y target vector
X=[]
y=[]

for x in images.keys():
    for z in images[x]:
        X.append(z)
        y.append(d[str(x)])

In [8]:
#one-hot encoding of y target
y=np.array(y)
y_ = np.zeros((y.size, y.max()+1))
y_[np.arange(y.size),y] = 1

In [9]:
#flatten feature vector into 1D vectors
X=[x.flatten() for x in X]
X=np.array(X)

In [10]:
#define number of inputs and outputs
num_inputs=224*192
num_outputs=2

In [11]:
#make weights and bias arrays
W = np.zeros((num_inputs, num_outputs))
b = np.zeros(num_outputs)

In [12]:
print(W.shape)
print(b.shape)
print(X.shape)

(43008, 2)
(2,)
(50, 43008)


In [13]:
#define net input of matrix which is dot product of feature matrix and weights
def net_input(X, W, b):
    return (X.dot(W) + b)

In [14]:
#define softmax function
def softmax(z):
    return (np.exp(z.T) / np.sum(np.exp(z), axis=1)).T

In [15]:
#define functions to convert to class labels
def to_classlabel(z):
    return z.argmax()

In [16]:
smax=softmax(net_input(X,W,b))

In [17]:
y_target=[to_classlabel(x) for x in y_]

In [18]:
W.shape

(43008, 2)

In [19]:
def cross_entropy(output, y_target):
    return - np.sum(np.log(output) * (y_target),axis=1)

In [20]:
np.sum(cross_entropy(smax,y_))

34.657359027997266

In [22]:
def cost(output, y_target):
    return np.mean(cross_entropy(output, y_target))

In [31]:
smax

[array([1.05241476e-311, 1.00000000e+000, 0.00000000e+000, 1.26051883e-269,
        6.23316054e-049, 0.00000000e+000]),
 array([1.68858434e-144, 1.00000000e+000, 3.17441556e-182, 2.65035028e-218,
        2.47572748e-031, 8.22077856e-240]),
 array([1.05601459e-272, 1.00000000e+000, 1.17746865e-255, 0.00000000e+000,
        1.05461011e-039, 4.48221948e-234]),
 array([3.07903291e-317, 1.00000000e+000, 2.74628502e-289, 3.48777696e-276,
        5.16199880e-075, 0.00000000e+000]),
 array([4.31655176e-198, 3.45442982e-048, 7.93923735e-266, 3.14972051e-192,
        1.00000000e+000, 0.00000000e+000]),
 array([3.89562857e-318, 1.00000000e+000, 2.33707990e-283, 7.76895874e-312,
        3.57400304e-025, 0.00000000e+000]),
 array([9.51116339e-271, 1.00000000e+000, 2.84763766e-225, 2.49819120e-240,
        5.46083894e-106, 1.11199359e-225]),
 array([1.09696265e-305, 1.00000000e+000, 6.46257914e-278, 0.00000000e+000,
        3.55096067e-069, 0.00000000e+000]),
 array([6.71241565e-247, 6.23864951e-008

In [23]:
np.array(X).shape

(150, 43008)